### Import

In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Poly'
excl_folders = ['NoFX_poly']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [3]:
len(dataset)

110040

### TRAIN SetNET

In [4]:
# model
setnet = models.SettingsNet(n_settings=dataset.max_num_settings).to(device)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

In [5]:
print(setnet)

SettingsNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (batchNorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (batchNorm4): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [7]:
# SAVE
models_folder = '../../saved/models'
model_name = '20201011_setnet_poly_best'
results_folder = '../../saved/results'
results_subfolder = '20201011_setnet_poly'

In [8]:
# TRAIN and TEST SetNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_settings_net(
        model=setnet,
        optimizer=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    # save models
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(setnet, '%s/%s' % (models_folder, model_name))
        print('--- saved best model ---')
        
    # append results    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[2000/79228 (3%)]	Total Loss: 9.6475	Avg Loss: 0.0048
Train Epoch: 0	[4000/79228 (5%)]	Total Loss: 13.3232	Avg Loss: 0.0033
Train Epoch: 0	[6000/79228 (8%)]	Total Loss: 15.5803	Avg Loss: 0.0026
Train Epoch: 0	[8000/79228 (10%)]	Total Loss: 17.3505	Avg Loss: 0.0022
Train Epoch: 0	[10000/79228 (13%)]	Total Loss: 18.8146	Avg Loss: 0.0019
Train Epoch: 0	[12000/79228 (15%)]	Total Loss: 20.2429	Avg Loss: 0.0017
Train Epoch: 0	[14000/79228 (18%)]	Total Loss: 21.4521	Avg Loss: 0.0015
Train Epoch: 0	[16000/79228 (20%)]	Total Loss: 22.5276	Avg Loss: 0.0014
Train Epoch: 0	[18000/79228 (23%)]	Total Loss: 23.5702	Avg Loss: 0.0013
Train Epoch: 0	[20000/79228 (25%)]	Total Loss: 24.5954	Avg Loss: 0.0012
Train Epoch: 0	[22000/79228 (28%)]	Total Loss: 25.5229	Avg Loss: 0.0012
Train Epoch: 0	[24000/79228 (30%)]	Total Loss: 26.3858	Avg Loss: 0.0011
Train Epoch: 0	[26000/79228 (33%)]	Total Loss: 27.2255	Avg Loss: 0.0010
Train Epoch: 0	[28000/79228 (35%)]	Total Loss: 28.0273	Avg Loss: 0.0010


In [9]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  75.56040793658808
Epoch:  48

Accuracy:  64.65243071331213
Epoch:  41

Accuracy:  64.04943656852053
Epoch:  41



In [10]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[41])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[41])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[41])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)